Grundlage:
* https://js.langchain.com/v0.1/docs/modules/chains/popular/sqlite/
* https://js.langchain.com/v0.1/docs/integrations/toolkits/sql/

* https://python.langchain.com/docs/tutorials/sql_qa/

# Setup

In [4]:
import sqlite3
import langchain
from langchain_community.utilities import SQLDatabase

In [5]:
db_path = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [6]:
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Chains
Sequence of steps that does the following:
* converts the question into a SQL query;
* executes the query;
* uses the result to answer the original question.


## Application state

In [7]:
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [8]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()

c:\Users\flemm\AppData\Local\pypoetry\Cache\virtualenvs\langchain-lernen-GE3QnVly-py3.11\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


In [9]:
!ollama list

NAME                     	ID          	SIZE  	MODIFIED     
qwen2.5-coder:7b         	2b0496514337	4.7 GB	9 hours ago 	
deepseek-r1:8b           	28f8fd6cdc67	4.9 GB	2 days ago  	
llama3.2:1b-instruct-q4_0	53f2745c8077	770 MB	3 months ago	
llama3.2:1b              	baf6a787fdff	1.3 GB	3 months ago	
llama3.1:8b              	42182419e950	4.7 GB	4 months ago	
mistral:instruct         	f974a74358d6	4.1 GB	4 months ago	


In [10]:
from langchain_ollama import ChatOllama

class FlemmingsChatOllama(ChatOllama):
    def with_structured_output(self, schema, *, include_raw=False, **kwargs):
        # Call bind_tools without the format parameter
        llm = self.bind_tools(tools=[schema])  # omit format="json"
        from langchain_core.output_parsers.pydantic import PydanticOutputParser
        from langchain_core.output_parsers.json import JsonOutputParser
        is_pydantic_schema = isinstance(schema, type)  # simplified check
        output_parser = (PydanticOutputParser(pydantic_object=schema)
                         if is_pydantic_schema else JsonOutputParser())
        # Compose the output parser
        from langchain_core.runnables import RunnableLambda
        parser_chain = RunnableLambda(lambda x: x) | output_parser
        # If include_raw is requested, add a passthrough (optional)
        if include_raw:
            from langchain_core.runnables.passthrough import RunnablePassthrough
            parser_assign = RunnablePassthrough.assign(parsed=lambda x: parser_chain.invoke(x),
                                                        parsing_error=lambda _: None)
            return llm | parser_assign
        else:
            return llm | parser_chain


In [11]:
import re
import json

def clean_response(raw_text: str) -> str:
    # Remove any header tokens like <|start_header_id|>assistant<|end_header_id|>
    cleaned = re.sub(r"<\|start_header_id\|>.*?<\|end_header_id\|>\s*", "", raw_text, flags=re.DOTALL)
    return cleaned.strip()

In [12]:
model = FlemmingsChatOllama(
    model=
        'llama3.2:1b',
    temperature=0
)

In [13]:
llm = ChatOllama(
    model=
        'llama3.2:1b',
    temperature=0
)

In [14]:
from typing_extensions import Annotated
class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]

In [15]:
def write_query_2(state: dict):
    prompt = query_prompt_template.invoke({
        "dialect": db.dialect,
        "top_k": 10,
        "table_info": db.get_table_info(),
        "input": state["question"],
    })
    # raw response
    raw_response = model.invoke(prompt).content
    # Clean header tokens
    cleaned = clean_response(raw_response)
    # wrap the plain text SQL query in a JSON object
    return {"query": cleaned}


In [21]:
def write_query(state: dict):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke({
        "dialect": db.dialect,
        "top_k": 10,
        "table_info": db.get_table_info(),
        "input": state["question"],
    })
    llm = model

    structured_llm = llm.with_structured_output(QueryOutput, method="json_schema")

    result = structured_llm.invoke(prompt)

    if not isinstance(result, dict):

        cleaned = clean_response(result)
        parsed = json.loads(cleaned)
        result = parsed
    return {"query": result["query"]}

In [20]:
write_query({"question": 
             "Give me the top 10 customers based on money spent."
             })

UnboundLocalError: cannot access local variable 'llm' where it is not associated with a value